In [1]:
import numpy as np
import math
import keras
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential, Model
#from keras.engine.training import collect_trainable_weights
from keras.layers import Conv2D,Dense, Flatten, Input, merge, Lambda
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K
import sys
import myenv
sys.path.append('../src/data/')
import data_preprocessing_from_yahoo_finance as dp

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
result_np = dp.data_preprocessing_1(ticker_list_input=['AAPL', 'PG'], path='../data/raw/')
c = myenv.myenv(result_np=result_np)

working on AAPL
working on PG


In [3]:
state = c.reset()

In [4]:
print(state[0].shape,state[1].shape)

(3,) (4, 2419, 3)


In [5]:
K.clear_session()
model = Sequential()
model.add(Conv2D(32, kernel_size=(100, 3),
                 #activation='relu',
                 data_format = 'channels_first',
                 input_shape=(4,2419,3)))
model.add(Conv2D(12, (20, 3)))
model.add(Flatten())
model.add(Dense(128,))
model.add(Dense(3, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# state[1] = state[1].reshape((1,) + state[1].shape)  # this is a Numpy array with shape (1, 3, 150, 150)

model.fit(state[1][None,:], state[0][None,:],batch_size=1,epochs=1)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[361608,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adadelta/gradients/dense_1/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@training/Adadelta/gradients/dense_1/MatMul_grad/MatMul"], transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](flatten_1/Reshape, training/Adadelta/gradients/dense_2/MatMul_grad/MatMul)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
model.predict(state[1][None,])